In [1]:
import pickle as pkl
import pandas as pd
import os

from helper_functions import formatTheTable, cleanTheTable, getTable


In [2]:
with open(f"data/error_lst2.pkl", "rb") as file:
    error_lst = pkl.load(file)

# 1. Systemize errors


## Run code and collect errors into a data.frame

In [3]:
new_error_dict = dict()
resultDF = pd.DataFrame()
for error_path in error_lst:
    brand = error_path.split('/')[2]
    model = error_path.split('/')[3].replace('.pkl', '')
    try:
        with open(error_path, "rb") as file:
            soup = pkl.load(file)
        tmp = getTable(soup)
        formated_df = formatTheTable(tmp)
        cleanDF = cleanTheTable(formated_df)
        cleanDF['Brand'] = brand
        cleanDF['Model'] = model
        resultDF = pd.concat([resultDF, cleanDF])
    except Exception as e:
        new_error_dict[error_path] = str(e)


error_df = pd.DataFrame({'paths':new_error_dict.keys(),
 'errors':new_error_dict.values()})

In [4]:
(error_df.
 assign(cnt = 1).
 groupby('errors')[['cnt']].
 sum().
 sort_values('cnt', ascending=False)
)

,cnt
errors,
'NoneType' object has no attribute 'find_all',54
list index out of range,15
"""['Schlagfläche', 'Länge', 'Rahmenhärte', 'Rahmenstärke', 'Schlägerfarbe', 'Besaitungshärte'] not in index""",5
"""['Schwunggewicht'] not in index""",5
"""['Gewicht (unbesaitet)', 'Balance (unbesaitet)', 'Schwunggewicht', 'Rahmenhärte', 'Material', 'Schlägerfarbe', 'Griffband', 'Besaitungshärte'] not in index""",2
"""['Gewicht (unbesaitet)', 'Balance (unbesaitet)'] not in index""",2
"""['Balance (unbesaitet)', 'Schwunggewicht', 'Rahmenhärte', 'Rahmenstärke'] not in index""",1
"""['Balance (unbesaitet)'] not in index""",1
"""['Gewicht (unbesaitet)'] not in index""",1


Makes sense to look into 'list index out of range' and "'NoneType' object has no attribute 'find_all'" errors

# 2. Fix most common errors

## 'list index out of range' erors fix

In [5]:
lor_df = error_df.loc[error_df.errors == 'list index out of range']
lor_df.head()

,paths,errors
0,data/htmls/Wilson/WilsonRF01PRO.pkl,list index out of range
5,data/htmls/Wilson/WilsonSix.One102Lite.pkl,list index out of range
7,data/htmls/Wilson/WilsonRF01Future.pkl,list index out of range
10,data/htmls/Wilson/WilsonRF01.pkl,list index out of range
13,data/htmls/Babolat/BabolatBoostStrike.pkl,list index out of range


In [6]:

def getTable2(soup):
    table = soup.find('tbody')
    rows = table.find_all('tr')
    # Initialize lists to store attributes and values
    attributes = []
    values = []
    # Iterate over each row and extract the attribute and value
    for row in rows:
        texts = row.find('td').text.split(":")
        if len(texts) < 2:
            next
        else:
            attribute = texts[0]
            value = texts[1]
            # Append to the lists
            attributes.append(attribute)
            values.append(value)
            df = pd.DataFrame({
                'Attribute': attributes,
                'Value': values
            })
    return df

def formatTheTable2(d):
    d2 = d.copy()
    d2.Attribute = d2.Attribute.str.replace(" ", "").str.strip()
    d3 = d2.set_index('Attribute').T
    
   
    return d3

def cleanTheTable2(d):
    tmp = d.copy()
    tmp = tmp.rename(columns = {'Swingweight':'SwingWeight'})
    tmp['HeadSize'] = tmp['HeadSize'].apply(lambda x: x.split('/')[0])
    tmp['Length'] =tmp['Length'].apply(lambda x: x.split('/')[1])
    tmp['StrungWeight'] =tmp['StrungWeight'].apply(lambda x: x.split('/')[1])
    tmp['Balance'] =tmp['Balance'].apply(lambda x: x.split('/')[1])
    tmp[['BeamWidth1', 'BeamWidth2', 'BeamWidth3']] = tmp['BeamWidth'].str.split(' / ', expand=True)
    tmp[['NumMains', 'NumCrosses']] = tmp['StringPattern'].str.split(' / ', expand=True)

    
    tmp[['MinStringTension', 'MaxStringTension']] = (tmp['StringTension'].
                                                     apply(lambda x: x.split(';')[1]).
                                                     str.
                                                     split('-', expand=True)
                                                    )
    cols = ['HeadSize', 'Length', 'StrungWeight', 'UnStrungWeight', 'SwingWeight',
            'Balance', 'Stiffness',
            'BeamWidth1', 'BeamWidth2', 'BeamWidth3',
            'NumMains', 'NumCrosses', 'MinStringTension', 'MaxStringTension'
            ]
    cols = [col for col in cols if col in tmp.columns]
    for col in cols:
        tmp[col] = tmp[col].str.extract(r'(\d+\.?\d*)').astype(float)
    tmp['StringDensity'] = tmp['HeadSize']/(tmp['NumMains']*tmp['NumCrosses'])
    return tmp[cols + ['StringDensity']]

In [7]:
resultDF2 = pd.DataFrame()
new_error_dict2 = dict()

In [8]:
for path in lor_df.paths:
    brand = path.split('/')[2]
    model = path.split('/')[3].replace('.pkl', '')
    try:
        with open(path, "rb") as file:
            soup = pkl.load(file)
        tmp = getTable2(soup)
        formated_df = formatTheTable2(tmp)
        cleanDF = cleanTheTable2(formated_df)
        cleanDF['Brand'] = brand
        cleanDF['Model'] = model
        resultDF2 = pd.concat([resultDF2, cleanDF])
    except Exception as e:
        new_error_dict2[path] = str(e)

In [9]:
resultDF2

Attribute,HeadSize,Length,StrungWeight,SwingWeight,Balance,Stiffness,BeamWidth1,BeamWidth2,BeamWidth3,NumMains,NumCrosses,MinStringTension,MaxStringTension,StringDensity,Brand,Model
Value,98.0,68.58,337.0,331.0,32.39,67.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01PRO
Value,98.0,68.58,295.0,301.0,33.02,64.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01Future
Value,98.0,68.58,318.0,319.0,32.64,64.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01
Value,115.0,70.10,283.0,315.0,35.56,69.0,28.0,27.0,27.0,16.0,19.0,21.0,27.0,0.378289,Volkl,V&#xf6;lklVostraV2
Value,100.0,68.58,278.0,290.0,33.99,69.0,22.5,24.5,22.5,16.0,18.0,22.0,27.0,0.347222,Volkl,V&#xf6;lklVostraV5
Value,98.0,68.58,337.0,327.0,31.98,65.0,20.5,22.5,20.5,16.0,19.0,22.0,27.0,0.322368,Volkl,V&#xf6;lklVostraV10(320g)
Value,98.0,68.58,318.0,314.0,33.48,68.0,20.5,22.5,21.0,16.0,19.0,22.0,27.0,0.322368,Volkl,V&#xf6;lklVostraV10(300g)
Value,100.0,68.58,323.0,322.0,31.98,61.0,22.0,23.0,20.0,18.0,20.0,20.0,25.0,0.277778,Prince,PrinceTourCarbon100P(305g)
Value,100.0,68.58,306.0,318.0,33.02,62.0,22.0,23.0,20.0,16.0,18.0,21.0,27.0,0.347222,Prince,PrinceTourCarbon100(290g)
Value,98.0,68.58,323.0,325.0,32.49,62.0,23.0,23.0,20.0,16.0,19.0,21.0,27.0,0.322368,Prince,PrinceTourCarbon98(305g)


In [10]:
new_error_dict2

{'data/htmls/Wilson/WilsonSix.One102Lite.pkl': "'HeadSize'",
 'data/htmls/Babolat/BabolatBoostStrike.pkl': "'HeadSize'",
 'data/htmls/Volkl/VˆlklPowerBridge10Mid.pkl': "'HeadSize'",
 'data/htmls/Volkl/V&#xF6;lklV-Cell8285gRacket.pkl': 'list index out of range',
 'data/htmls/ProKennex/ProKennexDestinyFCS(265g)Gold.pkl': "'HeadSize'"}

Seems like a code only works for English Language raquet data, but I'm ready to skip other five examples

## 'NoneType' object has no attribute 'find_all'

In [11]:
nt_df = error_df.loc[error_df.errors == "'NoneType' object has no attribute 'find_all'"]
nt_df.shape

(54, 2)

In [12]:
def getTable3(soup):
    table1 = soup.find_all(class_="SpecsDk")
    table2 = soup.find_all(class_="SpecsLt")
    table = [t.text for t in table1] + [t.text for t in table2]

    attributes = [t.split(":")[0] for t in table]
    values = [t.split(":")[1] for t in table]

    df = pd.DataFrame({
                'Attribute': attributes,
                'Value': values
            })
  

    return df

In [13]:
def cleanTheTable3(d):
    tmp = d.copy()
    tmp['HeadSize'] = tmp['HeadSize'].apply(lambda x: x.split('/')[0])
    tmp['Length'] =tmp['Length'].apply(lambda x: x.split('/')[1])
    tmp['StrungWeight'] =tmp['StrungWeight'].apply(lambda x: x.split('/')[0])
    tmp['UnStrungWeight'] =tmp['UnStrungWeight'].apply(lambda x: x.split('/')[0])
    tmp['Balance'] =tmp['Balance'].apply(lambda x: x.split('/')[0])
    tmp[['BeamWidth1', 'BeamWidth2', 'BeamWidth3']] = tmp['BeamWidth'].str.split(' / ', expand=True)
    tmp[['NumMains', 'NumCrosses']] = tmp['StringPattern'].str.split(' / ', expand=True)

    
    tmp[['MinStringTension', 'MaxStringTension']] = (tmp['StringTension'].
                                                     apply(lambda x: x.split('/')[0]).
                                                     str.
                                                     split('-', expand=True)
                                                    )
    cols = ['HeadSize', 'Length', 'StrungWeight', 'UnStrungWeight', 'SwingWeight',
            'Balance', 'Stiffness',
            'BeamWidth1', 'BeamWidth2', 'BeamWidth3',
            'NumMains', 'NumCrosses', 'MinStringTension', 'MaxStringTension'
            ]
    cols = [col for col in cols if col in tmp.columns]
    for col in cols:
        tmp[col] = tmp[col].str.replace(",", '.').str.extract(r'(\d+\.?\d*)').astype(float)
    tmp['StringDensity'] = tmp['HeadSize']/(tmp['NumMains']*tmp['NumCrosses'])
    return tmp[cols + ['StringDensity']]

In [14]:
resultDF3 = pd.DataFrame()
new_error_dict3= dict()

In [15]:
for path in nt_df.paths:
    brand = path.split('/')[2]
    model = path.split('/')[3].replace('.pkl', '')
    try:
        with open(path, "rb") as file:
            soup = pkl.load(file)
        tmp = getTable3(soup)
        formated_df = formatTheTable(tmp)
        cleanDF = cleanTheTable3(formated_df)
        cleanDF['Brand'] = brand
        cleanDF['Model'] = model
        resultDF3 = pd.concat([resultDF3, cleanDF])
    except Exception as e:
        new_error_dict3[path] = str(e)

In [16]:
tmp = getTable3(soup)
tmp

,Attribute,Value
0,Länge,"27in / 68,5cm"
1,Gewicht (unbesaitet),"300g/10,6oz"
2,Balance (unbesaitet),"31,5cm/9 pts HL"
3,Rahmenhärte,71
4,Material,40T Carbon/Graphite
5,Griffband,Solinco Synthetic
6,Besaitungshärte,22-27kg / 50-60lb
7,Schlagfläche,100 in / 645 cm
8,Gewicht,"320g / 11,3oz"
9,Balance,"32,51cm / 6 pts HL"


In [17]:
formated_df = formatTheTable(tmp)
formated_df

Attribute,HeadSize,Length,StrungWeight,UnStrungWeight,Balance,UnstrungBalance,SwingWeight,Stiffness,BeamWidth,Composition,RacketColours,GripType,StringPattern,StringTension
Value,100 in / 645 cm,"27in / 68,5cm","320g / 11,3oz","300g/10,6oz","32,51cm / 6 pts HL","31,5cm/9 pts HL",316,71,"23,5mm / 26mm / 23mm",40T Carbon/Graphite,Black,Solinco Synthetic,16 Mains / 19 CrossesMains skip,22-27kg / 50-60lb


In [18]:
cleanTheTable3(formated_df)

Attribute,HeadSize,Length,StrungWeight,UnStrungWeight,SwingWeight,Balance,Stiffness,BeamWidth1,BeamWidth2,BeamWidth3,NumMains,NumCrosses,MinStringTension,MaxStringTension,StringDensity
Value,100.0,68.5,320.0,300.0,316.0,32.51,71.0,23.5,26.0,23.0,16.0,19.0,22.0,27.0,0.328947


# Combine Additional Dataset

In [19]:
ad = pd.concat([resultDF3, resultDF2])

In [20]:
ad = ad.reset_index(drop=True)

In [21]:
ad.head()

Attribute,HeadSize,Length,StrungWeight,UnStrungWeight,SwingWeight,Balance,Stiffness,BeamWidth1,BeamWidth2,BeamWidth3,NumMains,NumCrosses,MinStringTension,MaxStringTension,StringDensity,Brand,Model
0,104.0,69.9,306.0,290.0,311.0,32.99,64.0,22.0,22.0,22.0,16.0,19.0,22.0,27.0,0.342105,Wilson,WilsonBlade104v9
1,97.0,68.5,283.0,270.0,301.0,33.02,67.0,23.5,23.5,23.5,16.0,19.0,22.0,27.0,0.319079,Wilson,WilsonProStaff97ULV14.0
2,98.0,68.5,326.0,310.0,327.0,31.75,60.0,24.0,24.0,24.0,16.0,20.0,22.0,27.0,0.306250,Wilson,WilsonClash98V2.0
3,100.0,68.5,275.0,260.0,286.0,33.99,68.0,24.0,26.5,24.5,16.0,19.0,21.0,25.0,0.328947,Wilson,WilsonUltra100ULV4.0
4,97.0,68.5,332.0,315.0,325.0,32.00,66.0,21.5,21.5,21.5,16.0,19.0,22.0,27.0,0.319079,Wilson,WilsonProStaff97V14.0


## Impute missing unstrung weight data

In [22]:
ad['WeightFromName'] = ad.Model.str.extract(r'\((\d+)g\)').astype(float)

In [23]:
ad.loc[ad.UnStrungWeight.isna(),'UnStrungWeight'] = ad.loc[ad.UnStrungWeight.isna(),'WeightFromName']

In [24]:
ad.loc[ad.UnStrungWeight.isna(),:]

Attribute,HeadSize,Length,StrungWeight,UnStrungWeight,SwingWeight,Balance,Stiffness,BeamWidth1,BeamWidth2,BeamWidth3,NumMains,NumCrosses,MinStringTension,MaxStringTension,StringDensity,Brand,Model,WeightFromName
43,98.0,68.58,337.0,NaN,331.0,32.39,67.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01PRO,NaN
44,98.0,68.58,295.0,NaN,301.0,33.02,64.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01Future,NaN
45,98.0,68.58,318.0,NaN,319.0,32.64,64.0,23.2,23.0,22.0,16.0,19.0,22.0,27.0,0.322368,Wilson,WilsonRF01,NaN
46,115.0,70.10,283.0,NaN,315.0,35.56,69.0,28.0,27.0,27.0,16.0,19.0,21.0,27.0,0.378289,Volkl,V&#xf6;lklVostraV2,NaN
47,100.0,68.58,278.0,NaN,290.0,33.99,69.0,22.5,24.5,22.5,16.0,18.0,22.0,27.0,0.347222,Volkl,V&#xf6;lklVostraV5,NaN


In [25]:
avg_dif = (ad.StrungWeight - ad.UnStrungWeight).mean()

In [26]:
ad.loc[ad.UnStrungWeight.isna(),'UnStrungWeight'] = ad.loc[ad.UnStrungWeight.isna(),'StrungWeight'] - avg_dif

In [27]:
ad.to_csv('data/additional_data.csv',index=False )

In [28]:
ad.head()

Attribute,HeadSize,Length,StrungWeight,UnStrungWeight,SwingWeight,Balance,Stiffness,BeamWidth1,BeamWidth2,BeamWidth3,NumMains,NumCrosses,MinStringTension,MaxStringTension,StringDensity,Brand,Model,WeightFromName
0,104.0,69.9,306.0,290.0,311.0,32.99,64.0,22.0,22.0,22.0,16.0,19.0,22.0,27.0,0.342105,Wilson,WilsonBlade104v9,NaN
1,97.0,68.5,283.0,270.0,301.0,33.02,67.0,23.5,23.5,23.5,16.0,19.0,22.0,27.0,0.319079,Wilson,WilsonProStaff97ULV14.0,NaN
2,98.0,68.5,326.0,310.0,327.0,31.75,60.0,24.0,24.0,24.0,16.0,20.0,22.0,27.0,0.306250,Wilson,WilsonClash98V2.0,NaN
3,100.0,68.5,275.0,260.0,286.0,33.99,68.0,24.0,26.5,24.5,16.0,19.0,21.0,25.0,0.328947,Wilson,WilsonUltra100ULV4.0,NaN
4,97.0,68.5,332.0,315.0,325.0,32.00,66.0,21.5,21.5,21.5,16.0,19.0,22.0,27.0,0.319079,Wilson,WilsonProStaff97V14.0,NaN
